# Improvement Trial With Transfer of Successful AI

This trial will test transferring AI from more successful AI to less successful ones.

In [1]:
import gpu_rec
from constants import *
from copy import deepcopy
from random import shuffle
from game import Game
import numpy as np
import sys

Using TensorFlow backend.


In [2]:
#game is now using lag=1 instead of lag=2 for q-score
#maybe this will help guide it to higher point values in short term, 
#with less opportunities for probabilities straying
test_config = deepcopy(DEFAULT_GAME_CONFIG)
test_config['memory'] = 1
test_config['discount'] = 0.01

init_game = Game(n_players=5, q_lag=1, config=test_config)
#goal: eventually train players to not want to grab more tickets

players = init_game.players
#former will be shuffled
player_list = [p for p in players]
overall_player_wins = []
overall_player_scores = []
overall_winning_scores = []
overall_game_turns = []



In [3]:
def which_min(x):
    min_val = min(x)
    return x.index(min_val)

def which_max(x):
    max_val = max(x)
    return x.index(max_val)

def aid_weakest_ai(players, wins):
    players[which_min(wins)].ai.receive_models_from_other_ai(players[which_max(wins)].ai)
    

In [4]:
temperatures = [ 0.2, 0.1, 0.1, 0.05, 0.01, 0.01, 0.01, 0.01, 0.001, 0.001, 0.001, 0.001, 0.00001, 0.00001, 0., 0., 0.]
n_trials_per_cycle = 200
n_cycles = 3

for temp_index, temperature in enumerate(temperatures):
    print '---TEMPERATURE: %.03f---' % temperature
    print '+IDX=%d+' % temp_index
    if temp_index==9:
        test_config['discount'] = 0.05
    elif temp_index==10:
        test_config['discount'] = 0.1
    elif temp_index==11:
        test_config['discount'] = 0.1
    elif temp_index==12:
        test_config['discount'] = 0.1
    elif temp_index==13:
        test_config['discount'] = 0.125
    test_config['temperature'] = temperature
    player_wins = [0 for _ in players]
    winning_scores = []
    player_scores = [[] for _ in players]
    game_turns = []
    for cycle in range(n_cycles):
        print 'in cycle %d' % cycle
        for i in range(n_trials_per_cycle):
            sys.stdout.write('.')
            shuffle(players)
            game = Game(pre_existing_players=players, config=test_config, separated_models=True)
            game.run(False)
            for i, player in enumerate(player_list):
                player_wins[i]+= player.win
                player_scores[i].append(player.total_points)
            winning_scores.append(game.winning_score)
            game_turns.append(game.turn)
        #don't train player 0, train player 1 partially, and train players 2 and 3 much more
        for i, player in enumerate(player_list):
            if i==1:
                player.ai.train_win()
                player.ai.train_q()
            elif i > 1:
                player.ai.train_win(20)
                player.ai.train_q(20)
    for player in player_list:
        player.ai.reset_history()
    aid_weakest_ai(player_list[1:], player_wins[1:])
    overall_player_wins.append(player_wins)
    overall_player_scores.append(player_scores)
    overall_winning_scores.append(winning_scores)
    overall_game_turns.append(game_turns)
    #print out some summaries
    print 'WINNING SCORE AVERAGE: %.1f' % np.mean(winning_scores)
    print 'NUMBER OF TURNS AVERAGE: %.1f' % np.mean(game_turns)
    for i, player in enumerate(player_list):
        print '---PLAYER %d STATS---' % i
        print 'WIN AVERAGE: %.3f' % (np.mean(player_wins[i])/(float(n_trials_per_cycle*n_cycles)))
        print 'SCORE AVERAGE: %.1f' % np.mean(player_scores[i])
        

---TEMPERATURE: 0.200---
+IDX=0+
in cycle 0
........................................................................................................................................................................................................in cycle 1
........................................................................................................................................................................................................in cycle 2
........................................................................................................................................................................................................WINNING SCORE AVERAGE: 27.7
NUMBER OF TURNS AVERAGE: 263.1
---PLAYER 0 STATS---
WIN AVERAGE: 0.035
SCORE AVERAGE: -28.3
---PLAYER 1 STATS---
WIN AVERAGE: 0.012
SCORE AVERAGE: -31.4
---PLAYER 2 STATS---
WIN AVERAGE: 0.050
SCORE AVERAGE: -20.6
---PLAYER 3 STATS---
WIN AVERAGE: 0.008
SCORE AVERAGE: -32.2
---PLAYER 4 STATS---
WIN AVERAG

ai.py:414: RuntimeWarning: invalid value encountered in divide
  return (selected)/np.sum(selected)
ai.py:386: RuntimeWarning: invalid value encountered in less
  return np.random.choice(len(new_probs), p=new_probs)


.......................................................................................................................................................................................................in cycle 1
........................................................................................................................................................................................................in cycle 2
........................................................................................................................................................................................................WINNING SCORE AVERAGE: 43.4
NUMBER OF TURNS AVERAGE: 227.1
---PLAYER 0 STATS---
WIN AVERAGE: 0.000
SCORE AVERAGE: -24.2
---PLAYER 1 STATS---
WIN AVERAGE: 0.288
SCORE AVERAGE: 26.5
---PLAYER 2 STATS---
WIN AVERAGE: 0.248
SCORE AVERAGE: 23.8
---PLAYER 3 STATS---
WIN AVERAGE: 0.223
SCORE AVERAGE: 25.5
---PLAYER 4 STATS---
WIN AVERAGE: 0.270
SCORE AVERAGE: 26.6
---TEMPERATURE: 0.0

........................................................................................................................................................................................................in cycle 1
........................................................................................................................................................................................................in cycle 2
........................................................................................................................................................................................................WINNING SCORE AVERAGE: 53.8
NUMBER OF TURNS AVERAGE: 218.1
---PLAYER 0 STATS---
WIN AVERAGE: 0.002
SCORE AVERAGE: -15.7
---PLAYER 1 STATS---
WIN AVERAGE: 0.295
SCORE AVERAGE: 33.3
---PLAYER 2 STATS---
WIN AVERAGE: 0.260
SCORE AVERAGE: 31.5
---PLAYER 3 STATS---
WIN AVERAGE: 0.258
SCORE AVERAGE: 31.1
---PLAYER 4 STATS---
WIN AVERAGE: 0.212
SCORE AVERAGE: 31.4
---TEMPERATURE: 0.

`temperature=0` no longer has the problems it used to have, and it seems safe to increase the discount value to some degree

In the next iteration I should continue using these AI files (they took like 40 hours to make) and continue increase the discount value and see how they behave

In [5]:
print game.turn
print i
print cycle
print temp_index
for player in players:
    print player
    print len(player.determine_possible_tracks())

227
4
2
16
player with 0/7 completed/uncompleted tickets, 15 train cards, 5 cars, and 
		-4 total points
42
player with 0/2 completed/uncompleted tickets, 14 train cards, 3 cars, and 
		10 total points
45
player with 0/3 completed/uncompleted tickets, 31 train cards, 2 cars, and 
		67 total points
29
player with 0/2 completed/uncompleted tickets, 7 train cards, 16 cars, and 
		50 total points
45
player with 0/2 completed/uncompleted tickets, 35 train cards, 8 cars, and 
		48 total points
73


In [6]:
np.mean(overall_winning_scores,1)

array([ 27.70833333,  37.55166667,  38.43166667,  41.37833333,
        43.44666667,  44.05833333,  44.065     ,  44.30333333,
        50.08666667,  51.33      ,  51.075     ,  51.37166667,
        53.81333333,  53.915     ,  55.265     ,  55.80333333,  54.91333333])

In [7]:
np.mean(overall_player_scores, ( 2,))

array([[-28.32833333, -31.42833333, -20.56166667, -32.24833333,  25.775     ],
       [-27.66      , -35.045     , -29.78166667,  24.88333333,
         23.11666667],
       [-30.44666667,  16.915     , -34.41166667,  20.99833333,
         16.13333333],
       [-30.05333333,  23.61666667,  24.75333333,  23.895     ,
         23.94666667],
       [-24.22666667,  26.45666667,  23.845     ,  25.50833333,
         26.59666667],
       [-25.295     ,  26.28666667,  26.52833333,  26.38333333,  26.09      ],
       [-24.05833333,  25.95      ,  25.39833333,  26.65      ,
         26.73833333],
       [-23.69      ,  27.185     ,  26.53666667,  26.02666667,
         25.86833333],
       [-17.40166667,  28.89833333,  30.01      ,  30.13666667,  31.38      ],
       [-17.58833333,  29.44      ,  30.25      ,  29.73666667,
         30.33833333],
       [-18.02666667,  28.64166667,  31.37833333,  29.455     ,
         29.53333333],
       [-17.71333333,  32.215     ,  29.29333333,  30.38833333,  30

In [8]:
for i, player in enumerate(player_list):
    player.ai.save_models('ai_h5/p%d_test09_%%s.h5' % i)

In [9]:
for player in player_list:
    print player

player with 0/7 completed/uncompleted tickets, 15 train cards, 5 cars, and 
		-4 total points
player with 0/2 completed/uncompleted tickets, 14 train cards, 3 cars, and 
		10 total points
player with 0/3 completed/uncompleted tickets, 31 train cards, 2 cars, and 
		67 total points
player with 0/2 completed/uncompleted tickets, 7 train cards, 16 cars, and 
		50 total points
player with 0/2 completed/uncompleted tickets, 35 train cards, 8 cars, and 
		48 total points


looks like the players are afraid to take tickets...
I will re-increase temperature next time but increase discount value. Maybe this will allow them to transition to "riskier" but better strategies if needed.